# 2020-01-22-numba-demo

## 1. This notebook

This demo of Awkward Array was presented on January 22, 2020, before the first stable version (1.0) was released. Some interfaces may have changed. To run this notebook, make sure you have version 0.1.87  ([GitHub](https://github.com/scikit-hep/awkward-1.0/releases/tag/0.1.87), [pip](https://pypi.org/project/awkward1/0.1.87/)) by installing

```bash
pip install 'awkward1==0.1.87'
```

before executing it in Jupyter (or include that release number in the Binder URL).

Depending on where you execute this notebook and how you installed or didn't install Awkward Array, you might need the following.

In [2]:
# The base of the GitHub repo is two levels up from this notebook.
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), "..", ".."))

## 2. Introduction to Awkward Array

Awkward Array is a library for manipulating data structures with NumPy-like idioms. For a core set of NumPy features—slicing, broadcasting, array-at-a-time operations, and such—it is a strict generalization from rectilinear arrays of numeric data types to unequal-width and heterogeneous lists and nested objects.

The name arose organically: these kinds of arrays are usually awkward to deal with.

### 2.1 Distinction from NumPy object arrays

Although NumPy arrays can contain arbitrary objects with `dtype('O')` type, those arrays can't be sliced or operated on with NumPy's usual idioms because they're really just pointers to pure Python objects.

In [3]:
import numpy as np
import awkward1 as ak

nparray = np.array([[1, 2, 3], [], [4, None, 5], [{"something": 1, "else": [2, 3]}]])
akarray = ak.Array([[1, 2, 3], [], [4, None, 5], [{"something": 1, "else": [2, 3]}]])

In [4]:
# NumPy can't slice the substructure of Python objects
nparray[2:, 0]

IndexError: too many indices for array

In [5]:
# Awkward can slice this substructure
akarray[2:, 0]

<Array [4, {something: 1, else: [2, 3]}] type='2 * ?union[int64, {"something": i...'>

In [6]:
# NumPy can't pass ufuncs into Python objects
np.sin(nparray)

TypeError: loop of ufunc does not support argument 0 of type list which has no callable sin method

In [7]:
# Awkward can pass ufuncs all the way down to the numericl data
np.sin(akarray)

<Array [[0.841, 0.909, 0.141], ... 0.141]}]] type='4 * var * ?union[float64, {"s...'>

In [8]:
# Here's a little more detail on the above:
ak.tolist(np.sin(akarray))

[[0.8414709848078965, 0.9092974268256817, 0.1411200080598672],
 [],
 [-0.7568024953079282, None, -0.9589242746631385],
 [{'something': 0.8414709848078965,
   'else': [0.9092974268256817, 0.1411200080598672]}]]

### 2.2 Columnar structure

Like NumPy (as well as [Apache Arrow](https://arrow.apache.org/) and [XND](https://xnd.io/)), Awkward Array operates on columnar arrays and prefers _O(1)_ views, rather than _O(n)_ computations (where _n_ is the number of elements in the array) wherever possible.

In [9]:
# Columnar structure of the above array
akarray.layout

<ListOffsetArray64>
    <offsets><Index64 i="[0 3 3 6 7]" offset="0" at="0x59194d151080"/></offsets>
    <content><IndexedOptionArray64>
        <index><Index64 i="[0 1 2 3 -1 4 5]" offset="0" at="0x59194d1550a0"/></index>
        <content><UnionArray8_64>
            <content index="0">
                <NumpyArray format="l" shape="5" data="1 2 3 4 5" at="0x59194d153090"/>
            </content>
            <content index="1">
                <RecordArray>
                    <field index="0" key="something">
                        <NumpyArray format="l" shape="1" data="1" at="0x59194d1590c0"/>
                    </field>
                    <field index="1" key="else">
                        <ListOffsetArray64>
                            <offsets><Index64 i="[0 2]" offset="0" at="0x59194d15b0d0"/></offsets>
                            <content><NumpyArray format="l" shape="2" data="2 3" at="0x59194d15d0e0"/></content>
                        </ListOffsetArray64>
                 

We started printing layout object representations in Pythonic `<angle brackets>` until we had to start nesting them, then XML seemed like an obvious generalization.

High-level data types are expressed in [Datashape](https://datashape.readthedocs.io/en/latest/) notation:

In [10]:
ak.typeof(akarray)

4 * var * ?union[int64, {"something": int64, "else": var * int64}]

with [extensions where necessary](https://github.com/blaze/datashape/issues/237). Similarly, these arrays will be portable to and from Apache Arrow (and other formats, if requested).

The idea is that Awkward Array provides **manipulation** capabilities, not **serialization** or **transport**.

### 2.3 Relevance for Numba

Numba, as you know, provides **computation** capabilities in a way that complements NumPy. Whereas NumPy requires array-at-a-time operations for performance, Numba enables imperative, pure Python code to have equal and often exceeding performance.

The analogy with Awkward is one-to-one:

|   | without Numba | with Numba |
|:-:|:-------------:|:----------:|
| **with NumPy** | array-at-a-time processing on numbers | general code on NumPy arrays and Python objects |
| **with Awkward** | array-at-a-time processing on data structures | general code on Awkward data structures |

The Awkward Array library includes Numba extensions with near feature parity: most operations that run outside of JIT-compiled functions run inside them as well.

In [11]:
import numba as nb

@nb.jit(nopython=True)
def run(array):
    out = np.empty(len(array), np.float64)
    for i in range(len(array)):
        out[i] = array[i]["x"]
        for y in array[i]["y"]:
            out[i] += y
    return out

akarray = ak.Array([{"x": 100, "y": [1.1, 2.2]}, {"x": 200, "y": []}, {"x": 300, "y": [3.3]}])

# Works for the layout classes, but not the high-level ak.Array wrapper yet.
run(akarray.layout)

array([103.3, 200. , 303.3])

Although Numba can take and return builtin Python objects (e.g. tuples, lists, dicts) and can let you define extensions for class instances with `@jitclass`, these objects need to be unboxed and boxed to Python types, which can be a bottleneck for large datasets. (At the very least, Python objects are a memory bottleneck!)

Since data in an Awkward Array are columnar, boxing and unboxing scales with the depth of the columnar layout, not the number of elements in the array. In this example, 4 array nodes were unboxed, though the array could have a million elements.

In [12]:
akarray.layout

<RecordArray>
    <field index="0" key="x">
        <NumpyArray format="l" shape="3" data="100 200 300" at="0x59194d18aa40"/>
    </field>
    <field index="1" key="y">
        <ListOffsetArray64>
            <offsets><Index64 i="[0 2 2 3]" offset="0" at="0x59194d18ca50"/></offsets>
            <content><NumpyArray format="d" shape="3" data="1.1 2.2 3.3" at="0x59194d18ea60"/></content>
        </ListOffsetArray64>
    </field>
</RecordArray>

<img src="img/example-hierarchy.png" style="width: 800px;">

In [13]:
%%timeit -n 1 -r 1

# Make an array of the same type with a million elements.
builder = ak.FillableArray()

for i in range(1000000):
    builder.beginrecord()
    builder.field("x")
    builder.integer(np.random.poisson(3) * 100)
    builder.field("y")
    builder.beginlist()
    for j in range(np.random.poisson(3)):
        builder.real(np.random.randint(5) * 1.1)
    builder.endlist()
    builder.endrecord()

akarray = builder.snapshot()
print(akarray, end="\n\n")

[{x: 300, y: [1.1, 3.3, 2.2]}, {x: 200, y: [2.2, ... {x: 400, y: [1.1, 0, 1.1]}]

29.9 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
run(akarray.layout)

array([103.3, 200. , 303.3])

We can even use Numba to build data structures with `FillableArray`, with a dramatic speedup (here, 50×).

In [15]:
%%timeit -n 1 -r 1

@nb.jit(nopython=True)
def build(builder):
    for i in range(1000000):
        builder.beginrecord()
        builder.field("x")
        builder.integer(np.random.poisson(3) * 100)
        builder.field("y")
        builder.beginlist()
        for j in range(np.random.poisson(3)):
            builder.real(np.random.randint(5) * 1.1)
        builder.endlist()
        builder.endrecord()
    return builder

print(ak.Array(build(ak.layout.FillableArray()).snapshot()), end="\n\n")

[{x: 400, y: [2.2, 1.1]}, {x: 400, y: [0, 0, ... {x: 400, y: [4.4, 3.3, 2.2, 0]}]

784 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


The equivalent in Numba is about as fast, though it has to box _O(million)_ lists and numbers.

In [16]:
%%timeit -n 1 -r 1

@nb.jit(nopython=True)
def build():
    outx = []
    outy = []
    for i in range(1000000):
        outx.append(np.random.poisson(3) * 100)
        tmp = []
        for j in range(np.random.poisson(3)):
            tmp.append(np.random.randint(5) * 1.1)
        outy.append(tmp)
    return (outx, outy)

outx, outy = build()
print(outx[:5])
print(outy[:5], end="\n\n")

[200, 200, 100, 400, 300]
[[2.2, 4.4, 2.2], [0.0, 0.0, 4.4], [4.4], [0.0, 0.0, 0.0], [2.2, 1.1, 4.4, 0.0, 1.1]]

1.05 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### 2.3 Why particle physics?

In our field, big datasets and nested data structures are ubiquitous. Nearly every physics analysis has to associate undiffentiated final-state particle trajectories to a hypothetical, hierarchical decay chain, such as this one:

<img src="img/ttbarHDecayDiagram_expanded.png" style="width: 800px;">

for billions of collision events.

Once the match has been made, the labeled particles could be represented by a rectilinear table. However, the task of looping through candidate combinations and uncertainties associated with pruning candidates _is the whole analysis, not a preprocessing step_. For most of the analysis, we are working with unequal-sized collections of objects (momentum vector components, energy, and other variables derived from detector measurements).

Our field has always had this problem. Even before Fortran had objects (or a multi-line `IF` statement), specialized physics software added the ability to operate on data structures. This is an exerpt from [Initiation to Hydra (1974)](https://cds.cern.ch/record/864527) by R.K. Böck, describing the concept of a non-numerical data structure to a physics audience.

<img src="img/hydra-2.png" style="width: 500px;">

Since the 1990's, object-oriented programming in C++ has been good for our field: it's natural to think of each particle as a C++ object with statically typed attributes, collected in variable-length `std::vector<Particle>`s.

**However,**

   * **we want to use Python:** last year marked a crossover threshold in which more physicist's GitHub repositories were [written in Python than C++](img/github-fraction.png),
   * **we still have huge datasets:** 10's of TB after considerable reduction (from the original 100's of PB).

NumPy would be good for our analysis scripts if it had more data types than rectilinear arrays of numbers. Pandas's [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html) is good for a single ragged dimension, but doesn't come close to what we need. Numba is excellent, but we need to get large datasets in and out efficiently. (Our [file format is already columnar](https://github.com/scikit-hep/uproot#readme); making intermediate Python objects would be a waste.)

Awkward Array was introduced to particle physicists in September 2019 and [is very popular in our community](../img/awkward-0-popularity.png). Emboldened by this response, I'm reimplementing it in a way that will make Numba, C++, and GPU integration easier to maintain.

### 2.4 Why not particle physics?

There's nothing domain-specific about nested data structures. The kinds of operations we want to do are [logical extensions of NumPy](https://github.com/jpivarski/2019-07-29-dpf-python/blob/master/03-columnar-data-analysis.ipynb) and can also be expressed as [per-array-item SQL](https://github.com/lgray/AwkwardQL#readme). Also wanting a general programming environment in Numba also has nothing, specifically, to do with particle physics.

There must be many other applications. How about this one?

In [18]:
!wget https://datahub.io/core/geo-countries/r/countries.geojson

--2020-01-21 16:24:22--  https://datahub.io/core/geo-countries/r/countries.geojson
Resolving datahub.io (datahub.io)... 104.24.113.103, 104.24.112.103, 2606:4700:3035::6818:7167, ...
Connecting to datahub.io (datahub.io)|104.24.113.103|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pkgstore.datahub.io/core/geo-countries/countries/archive/23f420f929e0e09c39d916b8aaa166fb/countries.geojson [following]
--2020-01-21 16:24:24--  https://pkgstore.datahub.io/core/geo-countries/countries/archive/23f420f929e0e09c39d916b8aaa166fb/countries.geojson
Resolving pkgstore.datahub.io (pkgstore.datahub.io)... 104.24.113.103, 104.24.112.103, 2606:4700:3036::6818:7067, ...
Connecting to pkgstore.datahub.io (pkgstore.datahub.io)|104.24.113.103|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24090863 (23M) [application/octet-stream]
Saving to: ‘countries.geojson’

countries.geojson   100%[===================>]  22.97M  3.01MB/s    in 13s     

In [19]:
countries = ak.Array("countries.geojson")
countries

<Array [... [30, -15.6], [30, -15.6]]]}}]}]] type='1 * var * {"type": string, "f...'>

In [20]:
ak.typeof(countries)

1 * var * {"type": string, "features": var * {"type": string, "properties": {"ADMIN": string, "ISO_A3": string}, "geometry": {"type": string, "coordinates": var * var * var * union[float64, var * float64]}}}

In [21]:
# longitude coordinates
countries["features", "geometry", "coordinates", :, :, :, :, :, 0]

<Array [... 29.8, 29.8, 29.9, 30, 30]]]]] type='1 * var * var * var * var * unio...'>

In [22]:
# latitude coordinates
countries["features", "geometry", "coordinates", :, :, :, :, :, 1]

<Array [... -15.6, -15.6, -15.6, -15.6]]]]] type='1 * var * var * var * var * un...'>

The fact that data analysts have managed so far with table-oriented tools like Pandas and SQL doesn't mean that they couldn't get more done if they had irregular data structures, too. Being able to write arbitrary algorithms in Numba on these big, irregular data structures further extends that reach.

## 3. The Awkward Array library in depth

### 3.1 Motivation for Awkward 1.0

The "0.x" version of Awkward (the one that is currently in use) is implemented entirely in NumPy. For performance, all of its algorithms must be expressed in a sequence of array-at-a-time calls, which led to [extremely clever case-by-case solutions](https://github.com/scikit-hep/awkward-array/blob/3442c51ed5dafb7d94f828c6cdc07659f9c03244/awkward/array/jagged.py#L1120-L1213).

In the end, though, we just can't generalize without being able to write `for` loops.

In [23]:
import awkward as oldak

oldarray = oldak.fromiter([[[0.0, 1.1, 2.2], [], [3.3, 4.4]], [[5.5]], [], [[6.6, 7.7, 8.8, 9.9]]])
newarray =       ak.Array([[[0.0, 1.1, 2.2], [], [3.3, 4.4]], [[5.5]], [], [[6.6, 7.7, 8.8, 9.9]]])

In [24]:
oldarray[:, ::-1, ::2]

NotImplementedError: this implementation cannot slice a JaggedArray in more than two dimensions

In [25]:
newarray[:, ::-1, ::2]

<Array [[[3.3], [], [0, ... [], [[6.6, 8.8]]] type='4 * var * var * float64'>

The original library had [awkward-numba](https://github.com/scikit-hep/awkward-array/tree/master/awkward-numba) and [awkward-cpp](https://github.com/scikit-hep/awkward-array/tree/master/awkward-cpp) subprojects, with the intention of adding an awkward-gpu, but the difficulty of maintaining them as independent implementations compounded.

### 3.2 Layered architecture

Awkward 1.0 is implemented in four layers:

   1. The `ak.Array` class and `ak.*` operations accessed by data analysts.
   2. The node objects that compose to form columnar data structures (in Python via pybind11).
   3. The C++ and Numba implementations of those nodes; reference-counted data structures that own memory.
   4. The CPU (and someday GPU) operations that navigate and fill arrays.

<img src="../img/awkward-1-0-layers.png" style="width: 500px;">

The spirit of NumPy—driving array-at-a-time operations from slow code with fast, precompiled kernels—is shifted down one layer: the C++ implementation is written without concern for speed, but only _O(1)_ operations are performed. The C++ is full of `std::shared_ptr` and `virtual` method calls, but _no loops over array data_ at this layer.

The Numba implementation, however, needs to be fast because Numba objects will be constructed in (the user's) loops over array data.

No memory allocation is performed in layer 4. It looks a lot like C code and has a pure C interface, so that C++ and Numba can both benefit from its implementations.

A walk through the code: how `__getitem__(tuple)` is implemented in

   * [old Awkward](https://github.com/scikit-hep/awkward-array/blob/master/awkward/array/jagged.py#L509-L779): lots of integer array tricks and `numpy.take`, valid for many special cases;
   * [new C++](https://github.com/scikit-hep/awkward-1.0/blob/master/src/libawkward/array/ListArray.cpp#L483-L620): recursive walk through the tuple, `dynamically_casting` class types and calling C kernels;
   * [new Numba](https://github.com/scikit-hep/awkward-1.0/blob/master/awkward1/_numba/array/listarray.py#L257-L490): recursive walk through the tuple, generating specialized code that calls C kernels;
   * [new C kernels](https://github.com/scikit-hep/awkward-1.0/blob/master/src/cpu-kernels/getitem.cpp#L288-L509): `for` loops on raw arrays.

Since the `__getitem__(tuple)` operation is actually defined in the C kernels, C++ and Numba are calling the same code when they run it.

In [27]:
@nb.jit(nopython=True)
def demo(array):
    return array[:, ::-1, ::2]

# Sorry: have to take "layout" out of its wrapper and wrap the result back up again...
ak.Array(demo(newarray.layout))

<Array [[[3.3], [], [0, ... [], [[6.6, 8.8]]] type='4 * var * var * float64'>

That way, implementations don't diverge.